In [2]:
using Pkg
# Pkg.activate(pwd())
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [324d7699] CategoricalArrays v0.10.2
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [a1dec852] Metida v0.12.4
  [ff71e718] MixedModels v4.6.0
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


#### calling packages

In [3]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels

#### using HighDimMixedModels

In [4]:
using HighDimMixedModels

┌ Info: Precompiling HighDimMixedModels [ced58d7a-d58c-464d-86d9-3a8462bebd28]
└ @ Base loading.jl:1317
  ** incremental compilation may be fatally broken for this module **



**Set up data and formula**

In [5]:
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


In [6]:
f = @formula(period ~ 0 + incid + hsz + (1|herd))

FormulaTerm
Response:
  period(unknown)
Predictors:
  incid(unknown)
  hsz(unknown)
  (herd)->1 | herd

### Construct object

#### parse random effect in formula

In [7]:
HMM = highDimMixedModel(f, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


__Check whether module load the data correctly__

In [8]:
@assert sum(HMM.M.M .!= df[:,:incid]) == 0
@assert sum(HMM.X.X .!= df[:,:hsz]) == 0
@assert size(HMM.Z.Z) == (56,15)

LoadError: AssertionError: sum(HMM.M.M .!= df[:, :incid]) == 0

### Optimization

#### MixedModel

In [8]:
ml = fit(MixedModel, @formula(period ~ 0 + incid + hsz +(1|herd)), df, REML = true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/zyxu/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Minimizing 16 	 Time: 0:00:00 (30.89 ms/it)


|             |    Est. |     SE |     z |      p | σ_herd |
|:----------- | -------:| ------:| -----:| ------:| ------:|
| incid       | -0.2939 | 0.0890 | -3.30 | 0.0010 |        |
| hsz         |  0.1317 | 0.0239 |  5.52 | <1e-07 |        |
| (Intercept) |         |        |       |        | 1.1569 |
| Residual    |  1.3496 |        |       |        |        |


In [14]:
show(ml)

Linear mixed model fit by REML
 period ~ 0 + incid + hsz + (1 | herd)
 REML criterion at convergence: 219.11119977822892

Variance components:
            Column   VarianceStd.Dev.
herd     (Intercept)  1.33834 1.15687
Residual              1.82132 1.34956
 Number of obs: 56; levels of grouping factors: 15

  Fixed-effects parameters:
─────────────────────────────────────────────
           Coef.  Std. Error      z  Pr(>|z|)
─────────────────────────────────────────────
incid  -0.293874   0.0890102  -3.30    0.0010
hsz     0.131742   0.0238714   5.52    <1e-07
─────────────────────────────────────────────

HighDimMixedModel

In [10]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

sigma = [1.0, 1.0]
negLog = 57.1099018030012
The initial object value is 57.1099018030012
sigma = [1.0, 1.0]
negLog = 57.1099018030012
OPTBL: starting point [1.0, 1.0]
sigma = [1.0, 1.0]
negLog = 57.1099018030012
sigma = [1.75, 1.0]
negLog = 55.97912076973617
sigma = [1.0, 1.75]
negLog = 52.37720093978967
sigma = [0.25, 1.0]
negLog = 64.50473446075652
sigma = [1.0, 0.25]
negLog = 128.22481961167495
sigma = [1.510389333579731, 1.4284710320133605]
negLog = 52.78019245866251
sigma = [0.8451353314889427, 1.437431307110896]
negLog = 53.411902302579605
sigma = [1.0, 1.9]
negLog = 52.31808747690741
sigma = [0.822599313455345, 1.9607062304338283]
negLog = 52.490735469672636
sigma = [1.048013829974836, 1.957616595969803]
negLog = 52.30520784011165
sigma = [1.0036098308862278, 2.0180590054596546]
negLog = 52.357982155798666
sigma = [1.0739875442712838, 1.9305681918459645]
negLog = 52.28417164563711
sigma = [1.1350679280066387, 1.887045936358205]
negLog = 52.251667221635145
sigma = [1.22890885574

In [11]:
betaM

1-element Vector{Float64}:
 -0.2938738291611581

In [12]:
betaX

1-element Vector{Float64}:
 0.1317420744001128